## Projekt: Rekonstrukcja kinematyki rozpadu Higgs → ττ za pomocą sieci neuronowej

### **Autor:** Jakub Kuśmierski  
### **Data:** 23.04.2025  
### **Opis:**   
Celem projektu jest stworzenie modelu regresyjnego o architekturze Fully Connected Neural Network (FCNN), którego zadaniem jest rekonstrukcja parametrów kinematycznych bozonu Higgsa oraz cząstek τ na podstawie danych symulacyjnych. 

Zazwyczaj problem wynika z faktu, że w rozpadzie taonów nie obserwujemy neutrin, które wynoszą ze zdarzenia część energii i pędu. Ich odtworzenie jest trudne w środowisku maszyny hadronowej, a MET (Missing Transverse Energy), który zawiera ich wkład, jest odtwarzany z dużą niedokładnością.

Model będzie oceniany poprzez porównanie jego wyników z obecnie stosowanym narzędziem referencyjnym fastMTT.


### Diagram Feynmana dla procesu Higgs → ττ

![Diagram Feynmanna dla procesu $H -> \tau \tau $](https://www.researchgate.net/publication/303921835/figure/fig2/AS:651132598751241@1532253517741/Feynman-diagram-describing-the-decay-H-t-t-with-one-t-subsequently-decaying.png)

## Definicja danych wejściowych i wyjściowych

### Dane wejściowe (cechy modelu)

- `dm1`, `dm2` – różnica masy (delta mass) w rozpadzie τ1 i τ2  
- `pt1`, `pt2` – pęd poprzeczny pierwszego i drugiego produktu rozpadu τ  
- `eta1`, `eta2` – pseudopospółrzędna η pierwszego i drugiego produktu  
- `phi1`, `phi2` – azymut pierwszego i drugiego produktu  
- `mass1`, `mass2` – masa widzialna produktów rozpadu (może to być np. układ hadronowy)  
- `trueMETx`, `trueMETy` – wygenerowane wartości brakującej energii poprzecznej (MET), komponenty w kierunkach x i y  
- `var_xx`, `var_yy` – wariancje niepewności MET w kierunkach x i y  
- `cov_xy` – kowariancja między komponentami x i y MET

###  Dane wyjściowe (predykcje modelu)

####  Kinematyka bozonu Higgsa:
- `H_m` – masa bozonu Higgsa  
- `H_pt` – pęd poprzeczny Higgsa  
- `H_eta` – pseudopospieszność η bozonu Higgsa  
- `H_phi` – azymut φ bozonu Higgsa  

####  Kinematyka cząstki τ1:
- `tau1_m` – masa cząstki τ1  
- `tau1_pt` – pęd poprzeczny τ1  
- `tau1_eta` – pseudopospieszność τ1  
- `tau1_phi` – azymut τ1  

####  Kinematyka cząstki τ2:
- `tau2_m` – masa cząstki τ2  
- `tau2_pt` – pęd poprzeczny τ2  
- `tau2_eta` – pseudopospspieszność τ2  
- `tau2_phi` – azymut τ2


Kinematyka τ1 i τ2, jeśli uda się wykonać, w głównej mierze chce skupić pracę na początek na kinmatyce bozonu Higgsa i dopiero później nad taonami.

**Uwaga dotycząca danych MET:**  
W projekcie planowane jest dodatkowe przetwarzanie danych MET (`trueMETx`, `trueMETy`), polegające na ich rozmyciu zgodnie z rozkładem normalnym (Gaussa), w celu zasymulowania realizmu rekonstrukcji eksperymentalnej. Rozkład ten może mieć zmienną wariancję dla każdej próbki, co pozwoli zwiększyć odporność modelu na przeuczenie względem statystyki MET oraz lepiej odzwierciedli warunki rzeczywistych eksperymentów hadronowych. Takie podejście ma na celu przygotowanie sieci do bardziej uniwersalnego działania w warunkach różnych rozrzutów niepewności.

## Lokalizacja danych

Dane do projektu znajdują się na scratch_cmsse, naleącym do doktoranta Wiktora Matyszkiewicza z Wydaiłu Fizyki Uniwersytetu Warszawskiego.

In [1]:
data_path = "/home/wmatyszk/scratch_cmsse/wmatyszk/project_sieci/csv/"

## Przewidywana architektura modelu

Model zastosowany w projekcie będzie oparty na architekturze **Fully Connected Neural Network (FCNN)**. Jego zadaniem będzie predykcja ciągłych parametrów kinematycznych bozonu Higgsa oraz cząstek τ, bazując na danych symulacyjnych z rekonstrukcją MET i obserwowanych produktach rozpadu.

#### **Etap bazowy – klasyczny FCNN**
Na pierwszym etapie projekt zakłada zastosowanie pojedynczej, w pełni połączonej sieci neuronowej o następującej strukturze:
- **Wejście:** wektor złożony z obserwowalnych parametrów rozpadu (pędy poprzeczne, kąty, masy, MET oraz jego wariancje i kowariancja)
- **Część ukryta:** kilka warstw Dense z funkcją aktywacji ReLU i regularizacją (np. Dropout), których liczba i szerokość zostaną ustalone eksperymentalnie
- **Wyjście:** 12-wymiarowy wektor z wartościami kinematycznymi Higgsa i obu taonów (masa, pT, eta, phi dla każdej z trzech cząstek)

Model ten będzie uczył się funkcji odwzorowującej dane wejściowe na pełną rekonstrukcję rozpadu Higgs → τ⁺τ⁻, przy czym trening będzie nadzorowany i oparty na rzeczywistych wartościach wygenerowanych w symulacji.

#### **Etap rozszerzony – architektura dwugałęziowa**
Na późniejszym etapie rozważone zostanie wdrożenie **dwugałęziowej architektury FCNN**, w której:
- **Gałąź pierwsza** przetwarza informacje dotyczące **rozpadu taonów** (parametry masy, pędu, kąta i masy produktów)
- **Gałąź druga** przetwarza parametry związane z **MET** (trueMETx, trueMETy, wariancje, kowariancja)
- Po przetworzeniu każdej z gałęzi, ich reprezentacje będą **łączone (concatenate)** i dalej przetwarzane w wspólnej części modelu w celu dokonania końcowej predykcji

Celem takiego rozwiązania jest umożliwienie modelowi osobnej specjalizacji w dwóch fizycznie różnych typach informacji – jednej związanej z widzialnymi cząstkami, drugiej z efektem nieobserwowalnych neutrin.